# Generate EAP for Allen morphologies - morphology 1

Note: this notebook can be run only once. To generate a new simulation you have to restart the kernel (Kernel->Restart)

In [ ]:
import neuron
from math import sin, cos
import numpy as np
import LFPy
import MEAutility as mu
import matplotlib.pyplot as plt
import neuroplotlib as npl
from pathlib import Path
import sys
from pprint import pprint
import os

In [ ]:
from axon_velocity.models import insert_biophysics, insert_simple_biophysics, \
    get_default_biophysics_params, planarize_swc, save_cell, create_mea_probe, center_cell_xy
from axon_velocity import plot_amplitude_map, plot_peak_latency_map

In [ ]:
%matplotlib notebook

In [ ]:
save_fig = True
save_results = True

In [ ]:
try:
    import neuron
except:
    print('NEURON is not installed.')

mechanism_folder = Path('..') / 'mechanisms'

if not neuron.load_mechanisms(str(mechanism_folder)):
    print('Compile mod files in the mechanisms/ folder: from the mechanisms/ folder, run nrnivmodl')

In [ ]:
# simple biophysiscs: dendrite - pas / soma/axon HH
# "complex" biophysics: dendrite - pas / soma - na + kv1 / axon - nax + kv1 
simple_biophysics = False

In [ ]:
params_dict = get_default_biophysics_params()
pprint(params_dict)

At this stage, one can also change the axial conductance (e.g. `sec.ra`), 
which likely affects the conduction velocity.

The `planar` variable decides wheter the z-axis is compressed (similar to a cell culture - `planar=True`) or the original morphology is used (`planar=False`).

In [ ]:
planar = True
z_offset = 5 # distance between cell plane and mea plane
zspan = 0

In [ ]:
morphology_dir = Path('..') / 'neuromorpho' / 'allen_cell_types'

morph_id = '538906745'
original_morphology_path = [m for m in morphology_dir.iterdir() if not 
                            m.name.startswith('.') and morph_id in str(m)][0]
if planar:
    morphology_path = planarize_swc(original_morphology_path, span_um=zspan)
else:
    morphology_path = original_morphology_path

In [ ]:
if save_fig:
    ax = npl.plot_neuron(morphology=str(morphology_path), plane='xy', color_axon='g')
    fig = ax.get_figure()
    fig_folder = Path('..') / 'figures'
    if not fig_folder.is_dir():
        os.makedirs(fig_folder)
    fig.savefig(fig_folder / 'allen1.pdf')    

In [ ]:
cell = LFPy.Cell(str(morphology_path), v_init=params_dict['v_init'], celsius=params_dict['celsius'],
                 Ra=params_dict['ra'], cm=params_dict['cm'], pt3d=True)

In [ ]:
# center in the xy plane
center_cell_xy(cell)

In [ ]:
ax_cell = npl.plot_neuron(cell, plane='xy', color_axon='g')

### Insert cell biophysics

Here we make the cell active by inserting biophysical mechanisms.

In [ ]:
if simple_biophysics:
    insert_simple_biophysics(cell)
else:
    insert_biophysics(cell, params_dict)

### Stimulating the cell

We can now add some stimulation. The stimulation can be a current clamp `iclamp` or synaptic inputs `syn`. The `stim_point` is where the cell will be stimulated (the closest cell segment to the `stim_point` is used).

In [ ]:
stim = 'syn' # or syn
# stimulate on the soma
stim_idx = cell.somaidx

syn_input_times = np.arange(2, 4)

syn_params = {'idx' : stim_idx,
              'e' : 0,                                # reversal potential
              'syntype' : 'ExpSyn',                   # synapse type
              'tau' : 2,                              # syn. time constant ms
              'weight' : 0.02,                         # syn. weight
              'record_current' : True                 # syn. current record
    }
clamp_params = {'idx' : stim_idx,
                'pptype' : 'IClamp',                   # IClamp point process
                'dur' : 300,                            # dur in ms
                'amp' : 2,                             # amp in nA
                'delay' : 5                            # delay in ms
    }

#%%

if stim == 'syn':
    synapse = LFPy.Synapse(cell, **syn_params)
    synapse.set_spike_times(np.array(syn_input_times))
else:
    clamp = LFPy.StimIntElectrode(cell=cell, **clamp_params)

In [ ]:
ax_cell.plot(cell.x[stim_idx], cell.y[stim_idx], 'o')

In [ ]:
if not planar:
    shift_pos = np.min(cell.z) - z_offset
    shift_neg = np.max(cell.z) + z_offset

    if np.abs(shift_pos) > np.abs(shift_neg):
        shift = shift_neg
    else:
        shift = shift_pos
else:
    shift = z_offset
    
print(f"z-position of MEA: {shift}")

### Define extracellular electrodes

Let's now define the extracellular electrodes using the [MEAutility](https://meautility.readthedocs.io/en/latest/) package.

In [ ]:
mea_dim = 100  # n rows x n cols
mea_pitch = 17.5  # rows and cols pitch
elec_size = 5

hdmea = create_mea_probe(pitch=mea_pitch, dim=mea_dim, elec_size=elec_size, z_offset=z_offset)

electrode = LFPy.RecExtElectrode(cell, probe=hdmea, n=10)

### Run the simulation

By passing the `electrode` argument `LFPy` also computes extracellular potentials. The `rec_vmem` argument allows to measure the membrane potenrtial at all segments.

In [ ]:
cell.simulate(probes=[electrode], rec_vmem=True)

In [ ]:
eap = electrode.data * 1000  # mV --> uV

### Plot membrane potentials

In [ ]:
soma_idx = cell.somaidx[0]
dend_idx = cell.get_closest_idx(-40, 100, 0)
axon_idx = cell.get_closest_idx(30, -400, 0)

In [ ]:
plt.figure()
plt.plot(cell.tvec, cell.vmem[soma_idx], label='soma')
plt.plot(cell.tvec, cell.vmem[dend_idx], label='dend')
plt.plot(cell.tvec, cell.vmem[axon_idx], label='axon')
plt.legend()

In [ ]:
# cutout single template
fs = 1 / cell.dt
ms_before = 2
ms_after = 10

min_chan, min_idx = np.unravel_index(np.argmin(eap), eap.shape)

In [ ]:
eap_cut = eap[:, min_idx - int(ms_before * fs): min_idx + int(ms_after * fs)]

In [ ]:
ax = mu.plot_mea_recording(eap_cut, hdmea, colors='gray')
npl.plot_neuron(cell, ax=ax, plane='xy', color='k', color_axon='g')

### Plot amplitude and peak latency map

In [ ]:
plot_amplitude_map(eap_cut, hdmea.positions, log=True)

In [ ]:
plot_peak_latency_map(eap_cut, hdmea.positions)

### Save templates and locations

In [ ]:
template = eap_cut
locations = hdmea.positions[:, :-1]  # save only x-y positions

In [ ]:
# save templates and locations
if save_results:
    data_folder = Path('..') / 'simulated_data' / 'allen'
    if planar:
        save_path = data_folder / f'allen1_planar_{zspan}um'
    else:
        save_path = data_folder / 'allen1_original'

    if not save_path.is_dir():
        os.makedirs(save_path)

    np.save(save_path / 'template.npy', template)
    np.save(save_path / 'locations.npy', locations)
    save_cell(cell, cell_name='allen1', save_folder=save_path)